## EyeON: Eye on Operational Networks
### a firmware inventory/threat analysis tool


In [1]:
from eyeon import observe
from pprint import pprint

### Objects
EyeON consists of two parts - an `observe` call and a `parse` call. `observe` works on a single file to return a suite of identifying metrics. `parse` calls `observe` recursively, returning an observation for each file in a directory. Both of these can be run either from a library import or a `CLI` command.

In [2]:
obs = observe.Observe("./tests/Obsidian.1.1.9.exe")
ols = observe.Observe("./tests/ls")

### Data Standard
Depending on the file type, e.g. PE or ELF, different observations will be collected. 
For instance, PE files typically contain more metadata and have signature information. Here we show some high-level characteristics, and we can dig into the certificates more thoroughly.

In [3]:
print("authentihash:", obs.authentihash)
print("filename:", obs.filename)
print("file magic:", obs.magic)
print("signature_validation:", obs.signatures[0]["verification"])

authentihash: e2f1f0c4b4b6524e1054bd5980129996bc021daa
filename: Obsidian.1.1.9.exe
file magic: PE32 executable (GUI) Intel 80386, for MS Windows, Nullsoft Installer self-extracting archive
signature_validation: True


In [4]:
pprint(obs.metadata)

{'FileInfo': {'CompanyName': 'Obsidian',
              'FileDescription': 'Obsidian',
              'FileVersion': '1.1.9',
              'LegalCopyright': 'Copyright © 2021 Dynalist Inc.',
              'ProductName': 'Obsidian',
              'ProductVersion': '1.1.9'},
 'OS': 'Windows',
 'dllRedirectionLocal': False,
 'peImport': ['KERNEL32.dll',
              'USER32.dll',
              'GDI32.dll',
              'SHELL32.dll',
              'ADVAPI32.dll',
              'COMCTL32.dll',
              'ole32.dll'],
 'peIsClr': False,
 'peIsDll': False,
 'peIsExe': True,
 'peLinkerVersion': '6.0',
 'peMachine': 'I386',
 'peOperatingSystemVersion': '4.0',
 'peSubsystem': 'WINDOWS_GUI',
 'peSubsystemVersion': '4.0'}


In [5]:
for sig in obs.signatures:
    print("digest algorithm:", sig["digest_algorithm"])
    print("digest value:", sig["sha1"])
    print("signers", sig["signers"])
    print("cert validation:", sig["verification"])
    for cert in sig["certs"]:
        pprint(cert)
        break
    break

digest algorithm: ALGORITHMS.SHA_1
digest value: e2f1f0c4b4b6524e1054bd5980129996bc021daa
signers SHA_1/RSA - C=GB, O=Sectigo Limited, CN=Sectigo Public Code Signing CA R36 - 4 auth attr - 2 unauth attr
cert validation: True
{'RSA_key_size': '4096 bits',
 'basic_constraints': 'CA=true',
 'cert._version': '3',
 'certificate_policies': 'Any Policy, ???',
 'expires_on': '2028-12-31 23:59:59',
 'ext_key_usage': 'Code Signing',
 'issued_on': '2021-05-25 00:00:00',
 'issuer_name': 'C=GB, ST=Greater Manchester, L=Salford, O=Comodo CA Limited, '
                'CN=AAA Certificate Services',
 'key_usage': 'Digital Signature, Key Cert Sign, CRL Sign',
 'serial_number': '48:FC:93:B4:60:55:94:8D:36:A7:C9:8A:89:D6:94:16',
 'sha256': '7db9f7402a523dd338663b33d55aad0dd08cb67e687df0d4d98e7c5fca27efd1',
 'signed_using': 'RSA with SHA-384',
 'subject_name': 'C=GB, O=Sectigo Limited, CN=Sectigo Public Code Signing Root '
                 'R46'}


There is also a Command Line component installed with the `eyeon` library containing 2 options: `eyeon observe` and `eyeon parse`.
`observe` generates output for a single file, whereas `parse` scans a directory.

It can be called as below (note `!` executes a terminal command):

In [6]:
! eyeon --help

usage: eyeon [-h] [-o OUTPUT_DIR] [-g LOG_FILE] [-v LOG_LEVEL]
             {observe,parse} ...

Eye on Operational techNology, an update tracker for OT devices

positional arguments:
  {observe,parse}       sub-command help
    observe             observe help
    parse               parse help

options:
  -h, --help            show this help message and exit
  -o OUTPUT_DIR, --output-dir OUTPUT_DIR
                        Path to results directory. Defaults to $pwd. Can set
                        on $EYEON_OUTPUT.
  -g LOG_FILE, --log-file LOG_FILE
                        Output file for log. If none, prints to console.
  -v LOG_LEVEL, --log-level LOG_LEVEL
                        Set the log level. Defaults to ERROR.


In [7]:
! eyeon --output-dir ./outputs observe ./tests/Obsidian.1.1.9.exe
! jq . ./outputs/Obsidian.*

{
  "bytecount": 72690816,
  "filename": "Obsidian.1.1.9.exe",
  "signatures": [
    {
      "certs": [
        {
          "cert._version": "3",
          "sha256": "7db9f7402a523dd338663b33d55aad0dd08cb67e687df0d4d98e7c5fca27efd1",
          "serial_number": "48:FC:93:B4:60:55:94:8D:36:A7:C9:8A:89:D6:94:16",
          "issuer_name": "C=GB, ST=Greater Manchester, L=Salford, O=Comodo CA Limited, CN=AAA Certificate Services",
          "subject_name": "C=GB, O=Sectigo Limited, CN=Sectigo Public Code Signing Root R46",
          "issued_on": "2021-05-25 00:00:00",
          "expires_on": "2028-12-31 23:59:59",
          "signed_using": "RSA with SHA-384",
          "RSA_key_size": "4096 bits",
          "basic_constraints": "CA=true",
          "key_usage": "Digital Signature, Key Cert Sign, CRL Sign",
          "ext_key_usage": "Code Signing",
          "certificate_policies": "Any Policy, ???"
        },
        {
          "cert._version": "3",
          "sha256": "33846b545a49c9be490